In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim

In [ ]:
class CreateDataset(Dataset):
    
    def __init__(self, img_dir, dataframe, transform=None):
        self.labels_frame = dataframe
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels_frame)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_frame.id[idx]) + ".jpg"
        image = Image.open(img_name)
        label = self.labels_frame.target[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return [image, label]

In [ ]:
# Read labels file
dataframe = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')

label_names = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')

test_frame = pd.DataFrame(data=label_names['id'])
test_frame['target'] = label_names['id']

labels = dataframe['breed']

submission_file = pd.DataFrame(index=label_names.index, columns=label_names.keys())
submission_file['id'] = label_names['id']

label_names.drop(['id'], axis=1, inplace=True)

code = range(len(label_names))

breed_to_code = dict(zip(label_names, code))

code_to_breed = dict(zip(code, label_names))

dataframe['target'] =  [breed_to_code[x] for x in dataframe.breed]

train_dir = '/kaggle/input/dog-breed-identification/train'
test_dir = '/kaggle/input/dog-breed-identification/test'

train_transform = transforms.Compose([transforms.Resize((256,256)),
                                      transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(5),
                                        transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

train_data = CreateDataset(train_dir, dataframe, train_transform)

test_transform = transforms.Compose([transforms.Resize((256,256)),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

test_data = CreateDataset(test_dir, test_frame, test_transform)

plt.imshow(train_data[10][0][0])
print(code_to_breed[train_data[10][1]])

print(train_data[11][1])

In [ ]:
# Specify data loaders

batch_size = 64
num_workers = 0
    
# obtain training indices that will be used for validation
valid_size = 0.2
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=1, num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
loaders_transfer = {'train':train_loader,'test':test_loader,'valid':valid_loader}

In [ ]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

# Specify model architecture 
model_transfer = models.vgg16(pretrained=True)

# Freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False
    
model_transfer.classifier[6] = torch.nn.Linear(model_transfer.classifier[6].in_features,120)

if use_cuda:
    model_transfer = model_transfer.cuda()
    
criterion_transfer = torch.nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(),lr=0.001)

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output,target)
            
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output,target)
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## Save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(
            valid_loss_min,valid_loss))
            torch.save(model.state_dict(),save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

In [ ]:
# train the model

n_epochs = 40

model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda,'model_transfer.pt')


In [ ]:
def test(loaders, model, submission, use_cuda):
    model.eval()
    for index, (data, _) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data = data.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output = nn.functional.softmax(output)
        output = np.squeeze(output).cpu().detach().numpy()
        submission.iloc[index, 1:121] = output

In [ ]:
if use_cuda:
    model_transfer.load_state_dict(torch.load('./model_transfer.pt'))
else:
    model_transfer.load_state_dict(torch.load('./model_transfer.pt', map_location=lambda storage, loc: storage))

In [ ]:
test(loaders_transfer, model_transfer, submission_file, use_cuda)

In [ ]:
submission_file.to_csv("submission.csv", index=False)